In [12]:
%matplotlib inline
import numpy as np
# import _pickle as cPickle
# import os
# import gzip
# from IPython.core.debugger import set_trace
# import scipy.optimize
import matplotlib.pyplot as plt
import tensorflow as tf

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

# Second exercise: CNN
In this exercise you will implement a CNN

In [13]:
# load data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
print("mnist {}".format(mnist))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
mnist Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f669bdb6e80>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f669bdb6d30>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f669bd02dd8>)


In [19]:
# define input variable
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

# initialize weights with a small amount of noise for symmetry breaking, and to prevent 0 gradients
# The generated values follow a normal distribution with specified mean and standard deviation, 
# except that values whose magnitude is more than 2 standard deviations from the mean are dropped and re-picked.
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

# define constant bias
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

# Computes a 2-D convolution given 4-D input and filter tensors. returning 4D tensor of same type as x
# ? means zero padding überall 2, oder ?
# X has 4D data format [batch, height, width, color channels]
# filter w has shape [filter_height, filter_width, in_channels, out_channels]  (no of filters, first colors...?)
# strides = The stride of the sliding window for each dimension of input 
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# first convolutional layer with 16 filters, one color
# initialise weights and bias (slightly randomized)
# 3 x 3, 1 input (only one color), 16 output channels
W_conv1 = weight_variable([3, 3, 1, 16])
b_conv1 = bias_variable([16])

# define tensor representing input layer: reshape flattened 748 Vector to 4d tensor 28 x 28 image matrices
# and the final dimension corresponding to the number of color channels.
x_image = tf.reshape(x, [-1, 28, 28, 1])

# convolve, same size (padding, stride...), reduce size to 14x14 by maxpooling
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# define second layer concolving same size, max-pooling --> 7x7   64 features ??
W_conv2 = weight_variable([3, 3, 16, 16])  # ??
b_conv2 = bias_variable([16])

# convolve, same size (padding, stride...), reduce size to 14x14 by maxpooling
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# we add a fully-connected layer with 128 neurons to allow processing on the entire image.
# We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, 
# and apply a ReLU.
W_fc1 = weight_variable([7 * 7 * 16, 128])
b_fc1 = bias_variable([128])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*16])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# implement dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# final fully connected layer
W_fc2 = weight_variable([128, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [27]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-2).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(200):
    batch = mnist.train.next_batch(200)
    if i % 10 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  print("test")
  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.155
step 10, training accuracy 0.765
step 20, training accuracy 0.855
step 30, training accuracy 0.86
step 40, training accuracy 0.92
step 50, training accuracy 0.93
step 60, training accuracy 0.925
step 70, training accuracy 0.945
step 80, training accuracy 0.945
step 90, training accuracy 0.945
step 100, training accuracy 0.95
step 110, training accuracy 0.945
step 120, training accuracy 0.955
step 130, training accuracy 0.965
step 140, training accuracy 0.965
step 150, training accuracy 0.955
step 160, training accuracy 0.95
step 170, training accuracy 0.98
step 180, training accuracy 0.965
step 190, training accuracy 0.955
test
test accuracy 0.9765
